In [26]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [35]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

template.invoke({"name": "Bob", "user_input": "What is your name?"})


ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')])

In [28]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [29]:
chain = template | model 

In [32]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'user_input': {'title': 'User Input', 'type': 'string'}}}

In [33]:
template.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'user_input': {'title': 'User Input', 'type': 'string'}}}

In [34]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

In [30]:
result = chain.invoke({"name": "Bob", "user_input": "What is your name?"})
print(f"==>> result: {result}")

==>> result: content='My name is Bob. How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-be63787c-241c-4d74-aa5d-d30e753dabdd-0'


In [19]:
type(result)

coroutine

In [23]:
from langchain_core.globals import set_debug

set_debug(False)

In [25]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

template2 = ChatPromptTemplate.from_messages(
    [
        ("human", "{x}"),
    ]
)


long_chain = template | model | output_parser | template2 | model
result = long_chain.batch(
    [
        {"name": "Bob", "user_input": "What is your name?"},
        {"name": "Minki", "user_input": "What is your last name?"},
    ]
)
print(f"==>> result: {result}")

==>> result: [AIMessage(content="Hello Bob, I was wondering if you could help me with some information on your company's products and services.", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 19, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9d7eabad-39e5-47a1-acd0-c5657160aa1d-0'), AIMessage(content='Is there anything specific you need assistance with?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 26, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-34f2dc3b-9db5-485a-94c1-c72b43f37441-0')]
